## Data Loading:

In [3]:
# torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as torchdata

# sklearn imports
from sklearn.utils import compute_sample_weight
from sklearn import metrics

# usual imports
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt

# loading dataset
from data_loader import TIHMDataset

In [4]:
# Change the path with the path of your dataset
DPATH = '../Dataset/'

In [5]:
train_dataset = TIHMDataset(
    root=DPATH, train=True, normalise='global', n_days=7
    )
#train data set

test_dataset = TIHMDataset(
    root=DPATH, train=False, normalise='global', n_days=7
    )
#test data set

In [6]:
print(f"The target names are:\n{train_dataset.target_names}")

The target names are:
['Blood pressure', 'Agitation', 'Body water', 'Pulse', 'Weight', 'Body temperature']


This dataset can be passed to a dataloader: (we will apply some pre-processing and define these again later)

Let us define the problem. We want to use the input data to predict there was agitation on a given day using the past 7 days worth of data. 

In [9]:
class AgitationDataset(torchdata.Dataset):
    def __init__(self, dataset):
        self.data, self.target = [], []
        
        for x, y in dataset:
            self.data.append(torch.tensor(x).float())#iterate through the provided dataset convert the input features into PyTorch tensors
            self.target.append(np.int64(y[-1, 1]>=1)) # the last of the days and agitation
        
        # also define the sample weight
        self.sw = compute_sample_weight(class_weight='balanced', y=self.target)

    def __getitem__(self, index):
        return self.data[index], self.target[index], self.sw[index]
    
    def __len__(self):
        return len(self.data)

In [10]:
train_agitation_dataset = AgitationDataset(train_dataset)
test_agitation_dataset = AgitationDataset(test_dataset)

In [11]:
#store the values in the PyTorch library
train_dl = torchdata.DataLoader(
    dataset=train_agitation_dataset, batch_size=1000, shuffle=True,
    )

test_dl = torchdata.DataLoader(
    dataset=test_agitation_dataset, batch_size=1000, shuffle=False,
    )

## LSTM

Define an LSTM model:

In [14]:
# simple LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_dim, sequence_length, hidden_size):
#input_dim, sequence_length, hidden_size???
        
        super().__init__()
#super()is used to copy the functions from another classes (i.e. nn.Module)
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.BatchNorm1d(num_features=sequence_length),#normalizes the outputs 
            nn.ReLU(),#introduces non-linearity
            )
        self.lstm = nn.LSTM(
            input_dim, hidden_size, batch_first=True,
            )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size, 2)
            )

    def forward(self, x):
        x = self.layer1(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :] # get the value from the last of the sequence
        x = self.layer2(x)
        return x

And now train:

In [16]:
def train(model, criterion, optimiser, train_loader, n_epochs):
    
    training_loss = []

    device = 'cuda' if torch.cuda.is_available() else 'cpu'#check to gpu
    model.to(device)

    def one_batch(x, y, sw):
        x, y, sw = x.to(device), y.to(device), sw.to(device)
        criterion.zero_grad()#resets the gradients
        output = model(x) # gets the output after passing the input data through the model.
        loss = criterion(output, y,)
        loss = (loss * sw / sw.sum()).sum() #computes the sum of the computed loss 
        loss.mean().backward() #computes the average loss 
        optimiser.step()
        return loss

    def one_epoch(train_loader):
        epoch_loss = []
        for batch in train_loader:
            loss = one_batch(*batch)
            epoch_loss.append(loss.item())
        return epoch_loss

    for epoch in tqdm.tqdm(range(n_epochs), desc='Training'):  #visual progress bar 
        epoch_loss = one_epoch(train_loader)
        training_loss.extend(epoch_loss)

    model.to('cpu')

    return training_loss

In [17]:
batch_x = next(iter(train_dl))[0]
lstm = LSTMModel(batch_x.shape[2], batch_x.shape[1], 200)

In [ ]:
training_loss = train(
    model=lstm, 
    criterion=nn.CrossEntropyLoss(reduction='none'), 
    optimiser=torch.optim.Adam(lstm.parameters(), lr=0.001),
    train_loader=train_dl,
    n_epochs=40,
    )

Training:  57%|██████████████████▉              | 23/40 [00:03<00:02,  6.41it/s]

And predict:

In [ ]:
def predict(model, test_loader):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    predictions = []
    targets = []

    for x, y, sw in test_loader:
        x, y, sw = x.to(device), y.to(device), sw.to(device)
        outputs = model(x)
        predictions.append(F.softmax(outputs, dim=1))
        targets.append(y)
    
    return torch.cat(predictions).to('cpu'), torch.cat(targets).to('cpu')

In [ ]:
predictions, targets = predict(lstm, test_dl)

In [ ]:
recall = metrics.recall_score(targets, predictions.max(dim=1).indices)
print(f"The recall was {recall*100:.2f}%")

In [ ]:
cfm = metrics.confusion_matrix(targets, predictions.max(dim=1).indices)
print(f"The confusion matrix is:\n{cfm}")
# TN   FP
# FN   TP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Compute confusion matrix
cfm = metrics.confusion_matrix(targets, predictions.max(dim=1).indices)

# Normalize the confusion matrix by dividing each value by the sum of the row (true labels)
cfm_percentage = cfm.astype('float') / cfm.sum(axis=1)[:, np.newaxis] * 100

# Create a ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(cfm_percentage, display_labels=['No Agitation', 'Agitation'])

# Plot confusion matrix with "Predicted" and "True" labels
fig, ax = plt.subplots(figsize=(6, 6))  # Adjust size as needed
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".2f")  # Display two decimal places

# Customize axis labels
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

# Add title
plt.title('LSTM Confusion Matrix with Percentages')
plt.show()


In [ ]:
sensitivity = cfm[1,1]/cfm[1,:].sum()
print(f"The sensitivity is: {sensitivity*100:.2f}%")

In [ ]:
specificity = cfm[0,0]/cfm[0,:].sum()
print(f"The specificity is: {specificity*100:.2f}%")

In [ ]:
accuracy = cfm.trace() / cfm.sum()  # trace() gives the sum of diagonal elements (true positives + true negatives)
print(f"The accuracy is: {accuracy*100:.2f}%")

In [ ]:
precision = cfm[1,1] / cfm[:,1].sum()  # True Positives / (True Positives + False Positives)
print(f"The precision is: {precision*100:.2f}%")

In [ ]:
def predict_proba(model, test_loader):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    predictions = []
    targets = []

    with torch.no_grad():
        for x, y, _ in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            predictions.append(F.softmax(outputs, dim=1)[:, 1].cpu().numpy())  # Probabilities for the positive class
            targets.append(y.cpu().numpy())
    
    return np.concatenate(predictions), np.concatenate(targets)


In [ ]:
probs, true_labels = predict_proba(lstm, test_dl)
from sklearn.metrics import roc_curve, auc

# Compute ROC curve values
fpr, tpr, _ = roc_curve(true_labels, probs)

# Calculate AUC
roc_auc = auc(fpr, tpr)
print(f"The AUC is: {roc_auc*100:.2f}%")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Obtain model predictions
probs, true_labels = predict_proba(lstm, test_dl)

# Get predicted labels
pred_labels = probs >= 0.5  # Convert probabilities to binary predictions

# Compute precision and recall
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)

# Calculate F1 score
f1 = f1_score(true_labels, pred_labels)
print(f"The F1 score is: {f1*100:.2f}%")

In [ ]:
class BLSTMModel(nn.Module):
    def __init__(self, input_dim, sequence_length, hidden_size, num_layers=1):
        super().__init__()
        
        # Layer 1: Linear and BatchNorm followed by ReLU
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.BatchNorm1d(num_features=sequence_length),
            nn.ReLU(),
        )
        
        # Bidirectional LSTM layer
        self.lstm = nn.LSTM(
            input_dim, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True
        )
        
        # Layer 2: Linear layer for output, considering bidirectional
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size * 2, 2)  # Multiply by 2 due to bidirectionality
        )

    def forward(self, x):
        # Pass through Layer 1
        x = self.layer1(x)
        
        # Pass through LSTM (outputs both hidden states for forward and backward directions)
        x, _ = self.lstm(x)
        
        # Take the last hidden state from both directions
        x = torch.cat((x[:, -1, :self.lstm.hidden_size], x[:, 0, self.lstm.hidden_size:]), dim=1)
        
        # Pass through Layer 2 for classification
        x = self.layer2(x)
        return x

In [ ]:
batch_x = next(iter(train_dl))[0]
blstm = BLSTMModel(batch_x.shape[2], batch_x.shape[1], 200)

training_loss = train(
    model=blstm, 
    criterion=nn.CrossEntropyLoss(reduction='none'), 
    optimiser=torch.optim.Adam(blstm.parameters(), lr=0.001),
    train_loader=train_dl,
    n_epochs=40,
)

In [ ]:
predictions, targets = predict(blstm, test_dl)

b_recall = metrics.recall_score(targets, predictions.max(dim=1).indices)
print(f"The recall was {b_recall*100:.2f}%")

bcfm = metrics.confusion_matrix(targets, predictions.max(dim=1).indices)
print(f"The confusion matrix is:\n{bcfm}")

In [ ]:
bcfm = metrics.confusion_matrix(targets, predictions.max(dim=1).indices)

# Normalize the confusion matrix by dividing each value by the sum of the row (true labels)
bcfm_percentage = bcfm.astype('float') / bcfm.sum(axis=1)[:, np.newaxis] * 100

# Create a ConfusionMatrixDisplay object
disp = ConfusionMatrixDisplay(bcfm_percentage, display_labels=['No Agitation', 'Agitation'])

# Plot confusion matrix with "Predicted" and "True" labels
fig, ax = plt.subplots(figsize=(6, 6))  # Adjust size as needed
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".2f")  # Display two decimal places

# Customize axis labels
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')

# Add title
plt.title('B-LSTM Confusion Matrix with Percentages')
plt.show()

In [ ]:
b_sensitivity = bcfm[1,1]/bcfm[1,:].sum()
print(f"The sensitivity is: {b_sensitivity*100:.2f}%")

In [ ]:
b_specificity = bcfm[0,0]/bcfm[0,:].sum()
print(f"The specificity is: {b_specificity*100:.2f}%")

In [ ]:
b_accuracy = bcfm.trace() / bcfm.sum()  # trace() gives the sum of diagonal elements (true positives + true negatives)
print(f"The accuracy is: {b_accuracy*100:.2f}%")

In [ ]:
b_precision = bcfm[1,1] / bcfm[:,1].sum()  # True Positives / (True Positives + False Positives)
print(f"The precision is: {b_precision*100:.2f}%")

In [ ]:
probs, true_labels = predict_proba(blstm, test_dl)
from sklearn.metrics import roc_curve, auc

# Compute ROC curve values
fpr, tpr, _ = roc_curve(true_labels, probs)

# Calculate AUC
b_roc_auc = auc(fpr, tpr)
print(f"The AUC is: {b_roc_auc*100:.2f}%")

In [ ]:
probs, true_labels = predict_proba(blstm, test_dl)

# Get predicted labels
pred_labels = probs >= 0.5  # Convert probabilities to binary predictions



# Calculate F1 score
b_f1 = f1_score(true_labels, pred_labels)
print(f"The F1 score is: {b_f1*100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define metrics names and values for two models
metrics_names = ['Precision', 'Recall', 'Sensitivity', 'Specificity', 'Accuracy', 'F1 Score', 'AUC']
model_a_values = [precision, recall, sensitivity, specificity, accuracy, f1, roc_auc]
model_b_values = [b_precision, b_recall, b_sensitivity, b_specificity, b_accuracy, b_f1, b_roc_auc]

# Define number of bars and their positions
bar_width = 0.35  # Width of the bars
index = np.arange(len(metrics_names))  # The label locations

# Create a figure
plt.figure(figsize=(10, 6))

# Plot bars for model A
plt.bar(index, model_a_values, bar_width, label='LSTM', color='skyblue')

# Plot bars for model B, shifted by bar_width
plt.bar(index + bar_width, model_b_values, bar_width, label='BLSTM', color='lightgreen')

# Add labels and formatting
plt.xlabel('Metrics')
plt.ylabel('Percentage')
plt.title('Comparison of Performance Metrics between Models')
plt.xticks(index + bar_width / 2, metrics_names, rotation=45, ha='right')  # Align x labels
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add legend
plt.legend()

# Adjust layout and show the plot
plt.tight_layout()
plt.show()